In [1]:
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import json
import time

In [4]:
ai = pd.read_csv('../data/processed/ai.csv')
meme = pd.read_csv('../data/processed/meme.csv')
gaming = pd.read_csv('../data/processed/gaming.csv')
rwa = pd.read_csv('../data/processed/rwa.csv')

In [10]:
ai = ai[ai['marketcap'] != -1]
meme = meme[meme['marketcap'] != -1]
gaming = gaming[gaming['marketcap'] != -1]
rwa = rwa[rwa['marketcap'] != -1]

In [49]:
ai_unique_cryptos = ai.groupby(['name', 'symbol', 'id']).size().reset_index(name='count')
ai_unique_cryptos = ai_unique_cryptos[[ 'id','name', 'symbol']]  

meme_unique_cryptos = meme.groupby(['name', 'symbol', 'id']).size().reset_index(name='count')
meme_unique_cryptos = meme_unique_cryptos[[ 'id','name', 'symbol']]  

gaming_unique_cryptos = gaming.groupby(['name', 'symbol', 'id']).size().reset_index(name='count')
gaming_unique_cryptos = gaming_unique_cryptos[[ 'id','name', 'symbol']]  


rwa_unique_cryptos = rwa.groupby(['name', 'symbol', 'id']).size().reset_index(name='count')
rwa_unique_cryptos = rwa_unique_cryptos[[ 'id','name', 'symbol']]  


In [ ]:
def obtener_logos(unique_cryptos):
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/info'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': '04839452-4370-4ff6-8aa9-f478a9edcd5f',
    }

    session = Session()
    session.headers.update(headers)

    image_links = []

    for symbol in unique_cryptos['symbol']:
        parameters = {
            'symbol': symbol,
            'aux': 'logo'  
        }
        
        try:
            response = session.get(url, params=parameters)
            response_data = json.loads(response.text)
            
            if 'data' in response_data and symbol in response_data['data']:
                logo_url = response_data['data'][symbol][0].get('logo', None)
            else:
                logo_url = None
            
            image_links.append({'symbol': symbol, 'image': logo_url})
            
            time.sleep(7)
            
        except (ConnectionError, Timeout, TooManyRedirects) as e:
            print(f"Error con el símbolo {symbol}: {e}")
            image_links.append({'symbol': symbol, 'image': None})

    image_df = pd.DataFrame(image_links)

    result = unique_cryptos.merge(image_df, on='symbol', how='left')
    
    return result


In [57]:
ai_logos = obtener_logos(ai_unique_cryptos)
meme_logos = obtener_logos(meme_unique_cryptos)
gaming_logos = obtener_logos(gaming_unique_cryptos)
rwa_logos = obtener_logos(rwa_unique_cryptos)

In [67]:
rwa_logos.head()

,id,name,symbol,image
0,2db6b38a-681a-4514-9d67-691e319597ee,Avalanche,AVAX,https://s2.coinmarketcap.com/static/img/coins/...
1,a9c04b71-33e4-4fed-8126-cf1612c7e198,Chainlink,LINK,https://s2.coinmarketcap.com/static/img/coins/...
2,1c1cd416-b027-4d73-9d4d-0a9edc63524d,Chromia,CHR,https://s2.coinmarketcap.com/static/img/coins/...
3,20a20296-c751-4011-88fa-fc2b8b0a8564,ELYSIA,EL,https://s2.coinmarketcap.com/static/img/coins/...
4,cb43529e-f4a4-4f18-a348-116ad35c9351,LTO Network,LTO,https://s2.coinmarketcap.com/static/img/coins/...


In [66]:
rwa_logos.tail()

,id,name,symbol,image
11,72ff9ca6-8231-40bd-ba05-e9758a2cbb4c,Stobox Token,STBU,https://s2.coinmarketcap.com/static/img/coins/...
12,54d246b7-f554-4386-97da-907a03c1a812,Synthetix,SNX,https://s2.coinmarketcap.com/static/img/coins/...
13,db7344b5-586a-4e9f-917f-1ce09831decd,TrueFi,TRU,https://s2.coinmarketcap.com/static/img/coins/...
14,ce222383-2753-44d4-b9ef-2dd67a7ea730,VeChain,VET,https://s2.coinmarketcap.com/static/img/coins/...
15,39b71592-9e16-4198-8ca5-3b19a3f41dd3,XDC Network,XDC,https://s2.coinmarketcap.com/static/img/coins/...


In [65]:
rwa_logos['image'][len(rwa_logos)-1]

'https://s2.coinmarketcap.com/static/img/coins/64x64/2634.png'

In [68]:
def actualizar_tamano_imagenes(df):
    df['image'] = df['image'].str.replace('64x64', '128x128', regex=False)
    return df


In [69]:
ai_logos = actualizar_tamano_imagenes(ai_logos)

In [73]:
meme_logos = actualizar_tamano_imagenes(meme_logos)
gaming_logos = actualizar_tamano_imagenes(gaming_logos)
rwa_logos = actualizar_tamano_imagenes(rwa_logos)

In [77]:
rwa_logos['image'][len(rwa_logos)-1]

'https://s2.coinmarketcap.com/static/img/coins/128x128/2634.png'

In [79]:
ai_logos.to_csv('../data/processed/ai_logos.csv', index=False)
meme_logos.to_csv('../data/processed/meme_logos.csv', index=False)
gaming_logos.to_csv('../data/processed/gaming_logos.csv', index=False)
rwa_logos.to_csv('../data/processed/rwa_logos.csv', index=False)

In [2]:
ai = pd.read_csv('../data/processed/ai.csv')
meme = pd.read_csv('../data/processed/meme.csv')
gaming = pd.read_csv('../data/processed/gaming.csv')
rwa = pd.read_csv('../data/processed/rwa.csv')

ai.shape, meme.shape, gaming.shape, rwa.shape

((114410, 11), (33454, 11), (126980, 11), (52833, 11))

In [3]:
ai_logo = pd.read_csv('../data/processed/ai_logos.csv')
meme_logo = pd.read_csv('../data/processed/meme_logos.csv')
gaming_logo = pd.read_csv('../data/processed/gaming_logos.csv')
rwa_logo = pd.read_csv('../data/processed/rwa_logos.csv')

ai_logo.shape, meme_logo.shape, gaming_logo.shape, rwa_logo.shape

((31, 4), (18, 4), (59, 4), (16, 4))

In [6]:
def agregar_logo(df, df_marketcap):
    # Leer el archivo CSV
    df_csv = df.copy()

    # Crear la nueva columna 'image' inicializada en una imagen por defecto
    df_csv['image'] = "https://cdn-icons-png.flaticon.com/512/272/272525.png"

    # Iterar sobre cada fila del DataFrame de marketcap
    for _, row in df_marketcap.iterrows():
        # Encontrar coincidencias en el CSV y actualizar el logo
        df_csv.loc[df_csv['id'] == row['id'], 'image'] = row['image']
    
    return df_csv

In [7]:
ai = agregar_logo(ai, ai_logo)
meme = agregar_logo(meme, meme_logo)
gaming = agregar_logo(gaming, gaming_logo)
rwa = agregar_logo(rwa, rwa_logo)

ai.shape, meme.shape, gaming.shape, rwa.shape

((114410, 12), (33454, 12), (126980, 12), (52833, 12))

In [9]:
ai.to_csv('../data/processed/ai.csv', index=False)
meme.to_csv('../data/processed/meme.csv', index=False)
gaming.to_csv('../data/processed/gaming.csv', index=False)
rwa.to_csv('../data/processed/rwa.csv', index=False)